In [1]:
import mediapipe as mp

mp_face_mesh = mp.solutions.face_mesh
FACEMESH_POINTS = mp_face_mesh.FACEMESH_TESSELATION  # all triangulated connections

In [3]:
FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 159, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 386, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282],

    # Nose bridge and tip
    "nose": [1, 2, 98, 327, 168, 197, 195, 5, 4, 45],
    "nose_bridge": [6, 197, 195, 5, 4],
    "nose_tip": [1, 2, 98, 327],

    # Lips (outer and inner)
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402, 317, 14, 87],
    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],

    # Chin outline (jawline)
    "chin": [152, 377, 400, 378, 379, 365, 397, 288,
             435, 367, 397, 365, 379, 378, 400, 377, 152,
             148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103],

    # Ears
    "left_ear": [234, 93, 132, 58, 172, 136],
    "right_ear": [454, 323, 361, 288, 397, 365],

    # Line between nose and lips (philtrum)
    "nose_to_lips": [2, 164, 0, 37, 267, 13],

    # Additional face outline (temple, jaw curve)
    "face_outline": [10, 338, 297, 332, 284, 251, 389,
                     356, 454, 323, 361, 288, 397, 365, 379,
                     378, 400, 377, 152, 148, 176, 149, 150,
                     136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103, 67, 109],

    # Mid-eye centers (for eyeball placement)
    "left_eye_center": [468],  # available if using iris model
    "right_eye_center": [473],
}
EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}
import numpy as np

def iris_center(iris_points, landmarks):
    pts = np.array([[landmarks[i].x, landmarks[i].y] for i in iris_points])
    return pts.mean(axis=0)
import cv2

def draw_feature(image, landmarks, feature_indices, color=(255,255,255), thickness=1):
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in feature_indices]
    for i in range(len(points)-1):
        cv2.line(image, points[i], points[i+1], color, thickness)

In [39]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize FaceMesh model (refine_landmarks=True gives iris points)
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=True,
    refine_landmarks=True,
    max_num_faces=1,
    min_detection_confidence=0.5
)

# Load image
image = cv2.imread('divija2.png')
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Process image to get landmarks
results = face_mesh.process(rgb_image)

# ---------------------------
# FACIAL FEATURE DICTIONARIES
# ---------------------------

FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282],
    "top_eye" : [414, 286, 258, 257, 259, 260, 467],  # upper eye contour
    "bottom_eye" : [453, 452, 451, 450, 449, 448],  # lower eye contour
    "top_eye_left": [190, 56, 28, 27, 29, 30, 247],
    "bottom_eye_left": [228, 229, 230, 231, 232, 233],
    # # Nose bridge and tip
    # "nose": [1, 2, 98, 327, 168, 197, 195, 5, 4, 45],
    # #"nose_bridge": [6, 197, 195, 5, 4],
    # "nose_tip": [1, 2, 98, 327],
    # Nose
    # "nose_bridge": [6, 197, 195, 5, 4],
    # "nose_outline_left": [98, 97, 2, 326],    # outer curve from bridge to nostril
    # "nose_outline_right": [327, 326, 2, 97],  # mirror outline
    # "nostril_left": [49, 50, 101, 118, 117],
    # "nostril_right": [279, 278, 330, 347, 348],
    # "nose_bottom": [97, 2, 326],  # base of nose

    # Lips
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],

    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],

    # Upper lip ridge (natural lip contour)
    "upper_lip": [61, 40, 37, 0, 267, 270, 409, 291],

    # Optional lower lip (for extra definition)
    "lower_lip": [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291],

    # Chin outline (jawline)
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],

    # # Ears
    # "left_ear": [234, 93, 132, 58, 172, 136],
    # "right_ear": [454, 323, 361, 288, 397, 365],

    # # Line between nose and lips (philtrum)
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],
    

    # Mid-eye centers (for eyeball placement)
    "left_eye_center": [468],  # available if using iris model
    "right_eye_center": [473],
}

# Iris (eyeball) indices (only exist when refine_landmarks=True)
EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# ---------------------------
# DRAWING HELPERS
# ---------------------------

def draw_feature(image, landmarks, indices, color=(255, 255, 255), thickness=1):
    """Draw line features (eyes, lips, etc.)"""
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in indices]
    for i in range(len(points) - 1):
        cv2.line(image, points[i], points[i + 1], color, thickness)


def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_indices, bottom_eyelid_indices, color=(255, 255, 255)):
    """
    Draw eyeball circle clipped at eyelids.
    - iris_indices: points of iris
    - top_eyelid_indices: upper eyelid contour
    - bottom_eyelid_indices: lower eyelid contour
    """
    # Iris points
    pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                    for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))

    # Eyelid bounds
    top_pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                        for i in top_eyelid_indices])
    bottom_pts = np.array([[landmarks[i].x * image.shape[1], landmarks[i].y * image.shape[0]] 
                           for i in bottom_eyelid_indices])
    y_top = np.min(top_pts[:, 1])
    y_bottom = np.max(bottom_pts[:, 1])

    # Draw circle as points only within eyelid bounds
    num_points = 100
    for theta in np.linspace(0, 2 * np.pi, num_points):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if y_top <= y <= y_bottom:
            cv2.circle(image, (x, y), 1, color, 1)  # small point to draw outline

    # Draw pupil at iris center
    pupil_radius = int(radius * 0.3)
    pupil_top = max(int(center[1] - pupil_radius), int(y_top))
    pupil_bottom = min(int(center[1] + pupil_radius), int(y_bottom))
    for y in range(pupil_top, pupil_bottom):
        for x in range(int(center[0] - pupil_radius), int(center[0] + pupil_radius)):
            if (x - center[0])**2 + (y - center[1])**2 <= pupil_radius**2:
                image[y, x] = color


# ---------------------------
# MAIN LOGIC
# ---------------------------

if results.multi_face_landmarks:
    for face_landmarks in results.multi_face_landmarks:
        # Draw facial line features
        for name, indices in FACIAL_FEATURES.items():
            draw_feature(image, face_landmarks.landmark, indices)

        for iris_name, iris_indices in EYEBALLS.items():
            if "left" in iris_name:
                draw_eyeball_clipped(image, face_landmarks.landmark, iris_indices,
                             FACIAL_FEATURES["top_eye_left"], FACIAL_FEATURES["bottom_eye_left"])
            else:
                draw_eyeball_clipped(image, face_landmarks.landmark, iris_indices,
                             FACIAL_FEATURES["top_eye"], FACIAL_FEATURES["bottom_eye"])

# Show output
cv2.imwrite("face.png", image)


I0000 00:00:1760856941.733401 48421607 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760856941.734935 48456174 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760856941.739918 48456180 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


True

In [93]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import scipy.ndimage

# Initialize MediaPipe
mp_face_mesh = mp.solutions.face_mesh
mp_selfie_segmentation = mp.solutions.selfie_segmentation

HAIR_MODEL_PATH = '/Users/divija/Divi Drive/workplace/Princeton/Sem 5/Carlab/carlab_2025/final_project/hair_segmenter.tflite'

# ---------------------------
# Simplified Facial Feature Dictionaries
# ---------------------------

FACIAL_FEATURES = {
    # Eyes (outline of left and right eyes)
    "left_eye": [33, 7, 163, 144, 145, 153, 154, 155, 133,
                 173, 157, 158, 470, 160, 161, 246],
    "right_eye": [263, 249, 390, 373, 374, 380, 381, 382,
                  362, 398, 384, 385, 475, 387, 388, 466],

    # Eyebrows
    "left_eyebrow": [70, 63, 105, 66, 107, 55, 65, 52, 53, 70],
    "right_eyebrow": [336, 296, 334, 293, 300, 276, 283, 282, 295, 285, 336],

    # Upper / lower eyelid contours for clipped eyeballs
    "top_eye": [414, 286, 258, 257, 259, 260, 467],
    "bottom_eye": [453, 452, 451, 450, 449, 448],
    "top_eye_left": [190, 56, 28, 27, 29, 30, 247],
    "bottom_eye_left": [228, 229, 230, 231, 232, 233],

    # Lips
    "outer_lips": [61, 146, 91, 181, 84, 17, 314, 405,
                   321, 375, 291, 308, 324, 318, 402,
                   317, 14, 87, 178, 88, 95, 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308],
    "inner_lips": [78, 95, 88, 178, 87, 14, 317, 402,
                   318, 324, 308, 291, 375, 321, 405, 314, 17, 84],
    "upper_lip": [61, 40, 37, 0, 267, 270, 409, 291],
    "lower_lip": [61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291],

    # Chin / jawline
    "chin": [58, 138, 172, 136, 150, 149, 176, 148, 152, 377, 400, 378, 379],
    # actual nose:
    "nose_left": [122, 196, 236, 198, 209, 49, 48, 64, 98],
    "nose_right": [420, 360, 278, 294, 327, 326],
    "nostrils_left" : [98, 240, 75, 60, 99, 97],
    "nostrils_right" : [370, 354, 458, 290],
    "nose_center": [238, 241,242,370, 354, 94],
    # Nose to lips (philtrum)
    "nose_to_lips_left": [203, 206, 216, 57],
    "nose_to_lips_right": [423, 426, 436, 287],

    # Mid-eye centers for eyeball placement
    "left_eye_center": [468],
    "right_eye_center": [473],
}   

# Iris (eyeball) indices
EYEBALLS = {
    "left_iris": [468, 469, 470, 471],
    "right_iris": [473, 474, 475, 476],
}

# ---------------------------
# Drawing helpers
# ---------------------------

def smooth_points(points, sigma=5):
    """
    Smooth a list of points using Gaussian smoothing independently on x and y.
    points: list of (x, y)
    sigma: Gaussian kernel standard deviation
    """
    
    points = np.array(points, dtype=np.float32)
    x_smooth = scipy.ndimage.gaussian_filter1d(points[:, 0], sigma=sigma)
    y_smooth = scipy.ndimage.gaussian_filter1d(points[:, 1], sigma=sigma)
    return list(zip(x_smooth.astype(int), y_smooth.astype(int)))
def draw_feature(image, landmarks, indices, color=(255, 255, 255), thickness=2, smooth=True):
    points = [(int(landmarks[i].x * image.shape[1]),
               int(landmarks[i].y * image.shape[0])) for i in indices]
    if smooth and len(points) >= 3:
        points = smooth_points(points, sigma=0.1)
    for i in range(len(points) - 1):
        cv2.line(image, points[i], points[i + 1], color, thickness)

def draw_eyeball_clipped(image, landmarks, iris_indices, top_eyelid_index, bottom_eyelid_index, color=(255, 255, 255)):
    """
    Draw an eyeball circle clipped at the top eyelid landmark.
    - iris_indices: list of iris landmarks
    - top_eyelid_index: single landmark index for upper eyelid to clip at
    - bottom_eyelid_indices: list of lower eyelid landmarks
    """
    H, W = image.shape[:2]
    
    # Iris points
    pts = np.array([[landmarks[i].x * W, landmarks[i].y * H] for i in iris_indices])
    center = np.mean(pts, axis=0)
    radius = np.mean(np.linalg.norm(pts - center, axis=1))
    
    # Upper eyelid landmark coordinate
    upper_y = landmarks[top_eyelid_index].y * H

    # Bottom eyelid bounds
    bottom_y = np.array([landmarks[bottom_eyelid_index].y * H])
    y_bottom = np.max(bottom_y)
    
    # Draw circle outline, skip points above upper eyelid landmark
    num_points = 200
    for theta in np.linspace(0, 2*np.pi, num_points):
        x = int(center[0] + radius * np.cos(theta))
        y = int(center[1] + radius * np.sin(theta))
        if upper_y <= y <= y_bottom:
            cv2.circle(image, (x, y), 1, color, 1)
    
    # Draw pupil clipped at the same bounds
    pupil_radius = int(radius * 0.3)
    pupil_top = max(int(center[1] - pupil_radius), int(upper_y))
    pupil_bottom = min(int(center[1] + pupil_radius), int(y_bottom))
    
    for y in range(pupil_top, pupil_bottom):
        for x in range(int(center[0] - pupil_radius), int(center[0] + pupil_radius)):
            if (x - center[0])**2 + (y - center[1])**2 <= pupil_radius**2:
                image[y, x] = color
# ---------------------------
# Main selective line drawing
# ---------------------------

def image_to_minimal_line_drawing_simplified(image_path, output_path):
    print(f"Processing: {image_path}")
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Image not found.")
        return
    H, W, _ = img.shape
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final_line_drawing = np.ones((H, W), dtype=np.uint8) * 255

    # FaceMesh
    with mp_face_mesh.FaceMesh(
        static_image_mode=True,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5
    ) as face_mesh:
        results_face = face_mesh.process(img_rgb)
        if results_face.multi_face_landmarks:
            face_landmarks = results_face.multi_face_landmarks[0]

    # Draw simplified facial features on final_line_drawing
    for name, indices in FACIAL_FEATURES.items():
        if "left_eye_center" in name or "right_eye_center" in name:
            continue
        draw_feature(final_line_drawing, face_landmarks.landmark, indices, color=0, thickness=2)

    # Draw clipped eyeballs on final_line_drawing
    for iris_name, iris_indices in EYEBALLS.items():
        if "left" in iris_name:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices,
                                160, 145, color=0)
        else:
            draw_eyeball_clipped(final_line_drawing, face_landmarks.landmark, iris_indices,
                                385, 374, color=0)

    # Selfie Segmentation for hair/body outline
    with mp_selfie_segmentation.SelfieSegmentation(model_selection=1) as segmentor:
        results_segmentation = segmentor.process(img_rgb)
        if results_segmentation.segmentation_mask is not None:
            person_mask = (results_segmentation.segmentation_mask > 0.5).astype(np.uint8) * 255
            blurred_outline = cv2.GaussianBlur(person_mask, (61,61), 0)
            edges = cv2.Canny(blurred_outline, 10, 50)
            edges = cv2.dilate(edges, np.ones((5,5),np.uint8), iterations=1)
            inverted = cv2.bitwise_not(edges)
            final_line_drawing = np.minimum(final_line_drawing, inverted)

    # Optional dedicated hair model
    if HAIR_MODEL_PATH:
        try:
            base_options = python.BaseOptions(model_asset_path=HAIR_MODEL_PATH, delegate=None)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img_rgb)
            options = vision.ImageSegmenterOptions(
                base_options=base_options,
                running_mode=vision.RunningMode.IMAGE,
                output_category_mask=True
            )
            with vision.ImageSegmenter.create_from_options(options) as segmenter:
                segmentation_result = segmenter.segment(mp_image)
                category_mask = segmentation_result.category_mask.numpy_view()
                hair_mask = np.where(category_mask == 1, 255, 0).astype(np.uint8)
                hair_mask = cv2.resize(hair_mask, (W,H), interpolation=cv2.INTER_NEAREST)
                blurred_hair = cv2.GaussianBlur(hair_mask, (41,41),0)
                edges = cv2.Canny(blurred_hair,10,50)
                edges = cv2.dilate(edges,np.ones((5,5),np.uint8),iterations=1)
                inverted = cv2.bitwise_not(edges)
                final_line_drawing = np.minimum(final_line_drawing, inverted)
        except Exception as e:
            print(f"Hair model error: {e}")
        
         # Save output
    cv2.imwrite(output_path, final_line_drawing)
    print(f"Saved simplified line drawing to: {output_path}")

# Example usage
image_to_minimal_line_drawing_simplified('divija_dhall.png', 'selective_line_drawing_simplified_divija_dhall.png')


Processing: divija_dhall.png


I0000 00:00:1760897963.864063 48421607 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760897963.866233 48889342 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760897963.874838 48889340 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760897963.887492 48421607 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760897963.889377 48889354 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1760897963.942955 48421607 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro
W0000 00:00:1760897963.945917 48889365 inference_feedback_manager.cc:114] Feedback manager requires 

Saved simplified line drawing to: selective_line_drawing_simplified_divija_dhall.png


In [62]:
import cv2
import numpy as np
from PIL import Image
from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation
import torch

# --- Configuration ---
IMAGE_PATH = "divija.png"
MODEL_CHECKPOINT = "mattmdjaga/segformer_b2_clothes"
# Map the label index (found in model config) to the target name
# For this specific model, Shirt/Top is often one of the first classes (e.g., 1 or 2).
# You may need to inspect the model's id2label mapping for the exact ID.
SHIRT_LABEL_ID = 4 # **Adjust this ID based on the model's actual mapping**
OUTPUT_OUTLINE_PATH = "shirt_outline_segformer.png"
OUTLINE_THICKNESS = 2
OUTLINE_COLOR = (0, 0, 0) # Black (BGR)
BACKGROUND_COLOR = (255, 255, 255) # White (BGR)

# --- 1. Load the Model and Processor ---
print("Loading SegFormer model and processor...")
processor = SegformerImageProcessor.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSemanticSegmentation.from_pretrained(MODEL_CHECKPOINT)

# --- 2. Load and Prepare the Image ---
image_pil = Image.open(IMAGE_PATH).convert("RGB")
image_cv = cv2.imread(IMAGE_PATH)
img_height, img_width, _ = image_cv.shape

# --- 3. Run Prediction / Segmentation ---
print("Running semantic segmentation...")
# Process image and get logits
inputs = processor(images=image_pil, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    
# ... (lines 33-36 of your script)
with torch.no_grad():
    outputs = model(**inputs)

# --- 3. Corrected Dimension Handling ---

# Outputs.logits shape is typically (1, C, H_model, W_model)
# We ensure the batch dimension (1) is present for interpolation
logits = outputs.logits.cpu().unsqueeze(0) if outputs.logits.dim() == 3 else outputs.logits.cpu()

# Check and fix if the batch dimension was implicitly removed
# If logits is just (C, H_model, W_model), we add the batch dimension.
if logits.dim() == 3:
    logits = logits.unsqueeze(0)
    
# Interpolate expects (N, C, H_model, W_model) and resizes to (N, C, H_img, W_img)
upsampled_logits = torch.nn.functional.interpolate(
    logits,
    size=(img_height, img_width),
    mode="bilinear",
    align_corners=False
)

# Get the predicted class index (ID) for each pixel.
# pred_mask will be (H_img, W_img)
pred_mask = upsampled_logits.squeeze(0).argmax(0)
# ... (rest of your script continues from here)
# --- 4. Extract Shirt Mask and Draw Outline ---
# Create a binary mask: 1 where pixel class = SHIRT_LABEL_ID, 0 everywhere else
shirt_mask = (pred_mask == SHIRT_LABEL_ID).numpy().astype(np.uint8) * 255 

if np.sum(shirt_mask) == 0:
    print("No shirt found based on the SegFormer model's class ID.")
    outline_image = np.full((img_height, img_width, 3), BACKGROUND_COLOR, dtype=np.uint8)
else:
    # Find contours
    contours, _ = cv2.findContours(shirt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create white background image
    outline_image = np.full((img_height, img_width, 3), BACKGROUND_COLOR, dtype=np.uint8)

    # Draw all found contours
    cv2.drawContours(outline_image, contours, -1, OUTLINE_COLOR, OUTLINE_THICKNESS)

# --- 5. Save the Output ---
print(f"Saving shirt outline to: {OUTPUT_OUTLINE_PATH}")
cv2.imwrite(OUTPUT_OUTLINE_PATH, shirt_mask)
print("Outline generation complete!")

Loading SegFormer model and processor...


Running semantic segmentation...
Saving shirt outline to: shirt_outline_segformer.png
Outline generation complete!
